## Intermodulation Products
Objectives:
Quantization: Observe how bit depth affects signal fidelity and the "staircase" effect.
SQNR: Validate the 
 dB rule.
Aliasing: Understand why the HackRF needs a specific sampling rate to see 16 MHz.
How to run the examples
To execute the Python script, click the 'Run This Cell' button above or press 'Shift + Enter'. The first time, make sure to execute the initialization part to load the required libraries.

Init Packages

In [ ]:
# Check if packages are installed and install if needed
import subprocess
import sys

def install_package(package):
    try:
        # Try to import the package
        __import__(package)
        print(f"{package} is already installed.")
    except ImportError:
        # If the package is not installed, install it
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of packages to check and install if needed
packages = ["numpy", "matplotlib", "ipywidgets", "scipy"]

for package in packages:
    install_package(package)

# Importing after ensuring packages are installed
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, VBox, HBox
from IPython.display import display, clear_output
from scipy.fft import fft, fftfreq, fftshift

## 1. Spurious Free Mixer Output

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import VBox, HBox, interactive_output

def plot_perfect_mixer_purple(f_rf, f_lo, p_rf, p_lo):
    # --- 1. Calculations ---
    f_if_diff = abs(f_rf - f_lo)
    f_if_sum = f_rf + f_lo
    
    # In a perfect mixer, output power equals RF power (0dB conversion loss)
    p_if = p_rf 

    # --- 2. Plotting ---
    plt.figure(figsize=(12, 6))
    
    def draw_spectral_line(f, p, color, label, marker='o'):
        plt.vlines(f, -100, p, colors=color, linewidth=4, label=label)
        plt.plot(f, p, marker, color=color, markersize=10)

    # Input Signals (Green and Blue)
    draw_spectral_line(f_rf, p_rf, '#2ecc71', f'RF Input ({f_rf} MHz)')
    draw_spectral_line(f_lo, p_lo, '#3498db', f'LO Input ({f_lo} MHz)')
    
    # Output Signals (Two shades of Purple)
    # Difference Frequency: Deep Purple
    draw_spectral_line(f_if_diff, p_if, '#6c3483', f'IF Output (Diff: |RF-LO| = {f_if_diff} MHz)')
    # Sum Frequency: Bright Purple/Lavender
    draw_spectral_line(f_if_sum, p_if, '#af7ac5', f'IF Output (Sum: RF+LO = {f_if_sum} MHz)')

    plt.title("Ideal Mixer", fontsize=14, fontweight='bold')
    plt.xlabel("Frequency (MHz)")
    plt.ylabel("Power (dBm)")
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.ylim(-100, max(p_lo, p_rf) + 15)
    plt.xlim(0, f_if_sum + 50)
    plt.legend(loc='upper right', frameon=True, shadow=True)
    
    # Text notes for students
    plt.text(f_if_diff, p_if + 5, 'Down-conversion', ha='center', color='#6c3483', fontweight='bold')
    plt.text(f_if_sum, p_if + 5, 'Up-conversion', ha='center', color='#af7ac5', fontweight='bold')

    plt.show()

# --- UI Controls ---
f_rf_s = widgets.IntSlider(min=100, max=1000, value=900, description='f_RF (MHz)')
f_lo_s = widgets.IntSlider(min=100, max=1000, value=830, description='f_LO (MHz)')
p_rf_s = widgets.IntSlider(min=-60, max=-10, value=-30, description='P_RF (dBm)')
p_lo_s = widgets.IntSlider(min=0, max=20, value=13, description='P_LO (dBm)')

ui = VBox([HBox([f_rf_s, f_lo_s]), HBox([p_rf_s, p_lo_s])])
out = interactive_output(plot_perfect_mixer_purple, {'f_rf': f_rf_s, 'f_lo': f_lo_s, 'p_rf': p_rf_s, 'p_lo': p_lo_s})

display(ui, out)

Output()

## 2. Stage Output with InterModulation Products

In [35]:
def plot_dynamic_nonlinear_analysis(f1, f2, p_out):
    # Ensure f1 is always the lower frequency for logic consistency
    f_low, f_high = min(f1, f2), max(f1, f2)
    
    # 2nd Order Products
    f_imd2_diff = f_high - f_low
    f_imd2_sum = f_low + f_high
    f_2f1, f_2f2 = 2*f_low, 2*f_high
    
    # 3rd Order Products
    f_imd3_diff_l = 2*f_low - f_high
    f_imd3_diff_h = 2*f_high - f_low
    f_imd3_sum_l = 2*f_low + f_high
    f_imd3_sum_h = 2*f_high + f_low
    f_3f1, f_3f2 = 3*f_low, 3*f_high
    
    # Power Levels
    p_imd2 = p_out - 40     
    p_imd3 = p_out - 55     

    plt.figure(figsize=(16, 9))
    
    def draw_tone(f, p, color, label, eqn, lw=4, ls='-'):
        if f > 0:
            plt.vlines(f, -140, p, colors=color, linewidth=lw, label=label, linestyles=ls)
            plt.plot(f, p, 'o', color=color, markersize=7)
            # Add equation text on top of the tone
            plt.text(f, p + 2, eqn, color=color, fontsize=10, 
                     ha='center', va='bottom', fontweight='bold', rotation=0)

    # 1. Fundamentals
    draw_tone(f_low, p_out, '#3498db', r'Fundamentals', r'$f_1$', lw=5)
    draw_tone(f_high, p_out, '#3498db', '', r'$f_2$')
    
    # 2. 2nd Order Family
    draw_tone(f_imd2_diff, p_imd2, '#f39c12', r'2nd Order Mixing', r'$f_2-f_1$')
    draw_tone(f_imd2_sum, p_imd2, '#f39c12', '', r'$f_1+f_2$')
    draw_tone(f_2f1, p_imd2, '#f1c40f', r'2nd Order Harmonics', r'$2f_1$', ls='--')
    draw_tone(f_2f2, p_imd2, '#f1c40f', '', r'$2f_2$', ls='--')
    
    # 3. 3rd Order Family
    draw_tone(f_imd3_diff_l, p_imd3, '#e74c3c', r'3rd Order Mixing Diff', r'$2f_1-f_2$')
    draw_tone(f_imd3_diff_h, p_imd3, '#e74c3c', '', r'$2f_2-f_1$')
    draw_tone(f_imd3_sum_l, p_imd3, '#c0392b', r'3rd Order Mixing Sum', r'$2f_1+f_2$')
    draw_tone(f_imd3_sum_h, p_imd3, '#c0392b', '', r'$2f_2+f_1$')
    draw_tone(f_3f1, p_imd3, '#943126', r'3rd Order Harmonics', r'$3f_1$', ls='--')
    draw_tone(f_3f2, p_imd3, '#943126', '', r'$3f_2$', ls='--')

    plt.title(f"Nonlinear Spectral Map with Equations: $f_1={f_low:.2f}$ MHz, $f_2={f_high:.2f}$ MHz", 
              fontsize=14, fontweight='bold')
    plt.xlabel("Frequency (MHz)")
    plt.ylabel("Power (dBm)")
    
    plt.xlim(0, max(f_3f2, f_imd3_sum_h) + 5)
    plt.ylim(-140, max(p_out + 20, 10)) # Increased upper limit for labels
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.legend(loc='upper right', frameon=True, fontsize='small', ncol=2)
    plt.show()

# --- UI Controls ---
f1_w = widgets.FloatSlider(min=1.0, max=20.0, value=5.0, step=0.01, description='f1 (MHz)')
f2_w = widgets.FloatSlider(min=1.0, max=20.0, value=6.0, step=0.01, description='f2 (MHz)')
p_out_w = widgets.IntSlider(min=-40, max=10, value=-10, description='P_fund (dBm)')

ui = VBox([HBox([f1_w, f2_w]), p_out_w])
out = interactive_output(plot_dynamic_nonlinear_analysis, {'f1': f1_w, 'f2': f2_w, 'p_out': p_out_w})

display(ui, out)

Output()

## 3. IP3 

The "Intercept Point" is a figure of merit used to characterize a device's linearity. It is the theoretical power level at which the distortion product equals the fundamental signal. In the **Third-Order Intercept (IP3)** as you saw in the simulation, for every \( 1\text{ dB} \) of input power increase, the \( IMD3 \) increases by \( 3\text{ dB} \).

For a device with gain \( G \) (in dB), the output power is:

$$
P_{out} = P_{in} + G
$$

However,  the OIP3 (Output IP3), is the point where the fundamental and \( IMD3 \) lines cross. If you measure \( P_{out} \) and the suppression \( \Delta_{IM3} \) (in dBc), you can find \( OIP3 \) using:

$$
OIP3 = P_{out} + \frac{\Delta_{IM3}}{2}
$$


In [47]:
def plot_ip3_corrected(gain, iip3, p_in_val):
    # --- 1. System Parameters ---
    oip3 = iip3 + gain
    
    # --- 2. Transfer Curve Data (for the left plot) ---
    p_in_axis = np.linspace(-60, 30, 100)
    p_out_fund_axis = p_in_axis + gain
    # The 3:1 slope rule: IMD3 power level
    p_out_imd3_axis = 3 * p_out_fund_axis - 2 * oip3
    
    # --- 3. Current Point Data (for the right plot) ---
    p_out_fund_val = p_in_val + gain
    # Corrected Physics: IMD3 grows 3dB per 1dB of Pin
    p_out_imd3_val = 3 * p_out_fund_val - 2 * oip3
    suppression = p_out_fund_val - p_out_imd3_val

    # --- 4. Plotting ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7), gridspec_kw={'width_ratios': [1.5, 1]})
    
    # LEFT PLOT: Transfer Characteristic
    ax1.plot(p_in_axis, p_out_fund_axis, label='Fundamental (Slope=1)', color='#3498db', lw=3)
    ax1.plot(p_in_axis, p_out_imd3_axis, label='IMD3 Product (Slope=3)', color='#e74c3c', lw=3)
    ax1.plot(iip3, oip3, 'ko', markersize=8) # The Intercept Point
    
    # Dynamic Markers
    ax1.vlines(p_in_val, -140, p_out_fund_val, colors='gray', linestyles=':', alpha=0.5)
    ax1.plot(p_in_val, p_out_fund_val, 'bo', markersize=8)
    ax1.plot(p_in_val, p_out_imd3_val, 'ro', markersize=8)
    
    # Update Title to include OIP3
    ax1.set_xlabel('Input Power ($P_{in}$) [dBm]')
    ax1.set_ylabel('Output Power ($P_{out}$) [dBm]')
    ax1.set_title(f'IP3 Transfer Characteristics (OIP3: {oip3:.2f} dBm)', fontweight='bold')
    ax1.grid(True, linestyle=':', alpha=0.6)
    ax1.set_xlim(-60, 35)
    ax1.set_ylim(-110, 65)
    ax1.legend(loc='lower right')

    # RIGHT PLOT: Spectrum Analyzer View
    f_fund = [10, 11]
    f_imd3 = [9, 12]
    
    # Fundamental Tones
    ax2.vlines(f_fund, -140, p_out_fund_val, colors='#3498db', lw=6)
    ax2.plot(f_fund, [p_out_fund_val]*2, 'bo', markersize=8)
    
    # IMD3 Tones (Correctly scaling now)
    ax2.vlines(f_imd3, -140, p_out_imd3_val, colors='#e74c3c', lw=5)
    ax2.plot(f_imd3, [p_out_imd3_val]*2, 'ro', markersize=8)
    
    # Measurement Arrow
    if suppression > 0:
        ax2.annotate('', xy=(11.5, p_out_imd3_val), xytext=(11.5, p_out_fund_val),
                     arrowprops=dict(arrowstyle='<->', color='green', lw=2))
        ax2.text(11.7, (p_out_fund_val + p_out_imd3_val)/2, f'Δ = {suppression:.1f} dBc', 
                 color='green', fontweight='bold', va='center')

    # Power Labels
    ax2.text(10.5, p_out_fund_val + 3, f'{p_out_fund_val:.1f} dBm', ha='center', color='#3498db', fontweight='bold')
    ax2.text(9, p_out_imd3_val + 3, f'{p_out_imd3_val:.1f} dBm', ha='center', color='#e74c3c', fontweight='bold')

    ax2.set_xlim(8, 14)
    ax2.set_ylim(-110, 65)
    ax2.set_xticks([9, 10, 11, 12])
    ax2.set_xticklabels(['$2f_1-f_2$', '$f_1$', '$f_2$', '$2f_2-f_1$'])
    ax2.set_ylabel('Power [dBm]')
    ax2.set_title(f'Spectrum Result (Pin: {p_in_val:.1f} dBm)', fontweight='bold')
    ax2.grid(True, linestyle=':', alpha=0.4)

    plt.tight_layout()
    plt.show()

# --- UI Controls ---
gain_w = widgets.IntSlider(min=0, max=30, value=15, description='Gain (dB)')
iip3_w = widgets.IntSlider(min=0, max=50, value=25, description='IIP3 (dBm)')
pin_w = widgets.FloatSlider(min=-50, max=10, value=-20, step=0.5, description='Pin (dBm)')

ui = VBox([HBox([gain_w, iip3_w]), pin_w])
out = interactive_output(plot_ip3_corrected, {'gain': gain_w, 'iip3': iip3_w, 'p_in_val': pin_w})

display(ui, out)



Output()

# 4. Cascaded IP3

In a receiver chain, each stage contributes non-linear distortion. As signals pass through amplifiers and mixers, they are amplified, increasing the likelihood of driving subsequent stages into nonlinearity.  The intercept point formulas must be calculated using linear power (Watts or mWatts) rather than logarithmic units (dBm).  The total system **IIP3** is calculated by summing the reciprocal of each stage's intercept point, scaled by the gain of the preceding stages. For a **5-stage system**:

$$
\frac{1}{IIP3_{total}} = \frac{1}{IIP3_1} + \frac{G_1}{IIP3_2} + \frac{G_1 G_2}{IIP3_3} + \frac{G_1 G_2 G_3}{IIP3_4} + \frac{G_1 G_2 G_3 G_4}{IIP3_5}
$$

### 3. Key Observations for Design

- **Gain-Linearity Trade-off**: Increasing the Gain ($G$) of an early stage (like an **LNA**) improves the Noise Figure, but it degrades the total **IIP3**. This happens because a higher **$G_1$** amplifies the distortion contributions.
  
- **Passive Stages**: Components like filters (S1 or S4 in your code) often have very high **IIP3** (e.g., 100 dBm). Their contribution to the reciprocal sum becomes nearly zero, essentially "passing through" the linearity of the previous stage minus their insertion loss.

- **The Bottleneck**: The stage where the **IIP3** curve drops most sharply is your linearity bottleneck. To improve the system's **IIP3**, you must either reduce the gain of the preceding stage or select a component with a higher individual **IIP3** for that position.


In [61]:
def dbm_to_mw(dbm):
    return 10**(dbm / 10.0)

def mw_to_dbm(mw):
    # Handle cases where mw might be exceptionally small/zero
    return 10 * np.log10(max(mw, 1e-15))

def calculate_cascade(g_list, iip3_list):
    # Convert dB/dBm to linear
    g_lin = [10**(g/10.0) for g in g_list]
    iip3_lin = [dbm_to_mw(i) for i in iip3_list]
    
    # Calculate reciprocal IIP3 contributions
    # 1/IIP3_total = 1/IIP3_1 + G1/IIP3_2 + (G1*G2)/IIP3_3 ...
    inv_iip3_total = 0
    cumulative_gain = 1.0
    contributions = []
    
    for i in range(len(g_list)):
        term = cumulative_gain / iip3_lin[i]
        inv_iip3_total += term
        contributions.append(term)
        cumulative_gain *= g_lin[i]
        
    iip3_total_mw = 1 / inv_iip3_total
    iip3_total_dbm = mw_to_dbm(iip3_total_mw)
    
    return iip3_total_dbm, contributions

def update_display(g1, i1, g2, i2, g3, i3, g4, i4, g5, i5):
    gains = [g1, g2, g3, g4, g5]
    iip3s = [i1, i2, i3, i4, i5]
    names = ['Filter/Ant', 'LNA', 'Mixer', 'IF Amp', 'ADC Driver']
    
    total_iip3, _ = calculate_cascade(gains, iip3s)
    
    # --- Visualization ---
    plt.figure(figsize=(10, 6))
    
    # Calculate cumulative IIP3 degradation stage by stage
    cumulative_results = []
    for i in range(1, 6):
        res, _ = calculate_cascade(gains[:i], iip3s[:i])
        cumulative_results.append(res)
        
    plt.plot(names, cumulative_results, marker='s', lw=3, color='#2c3e50', 
             markersize=10, markerfacecolor='#e74c3c')
    
    # Add labels to each point for clarity
    for i, val in enumerate(cumulative_results):
        plt.text(i, val + 1.5, f"{val:.2f} dBm", ha='center', fontweight='bold')

    plt.title(f"Cumulative System IIP3: {total_iip3:.2f} dBm", fontsize=14, fontweight='bold', color='#c0392b')
    plt.ylabel("IIP3 (dBm)", fontsize=12)
    plt.xlabel("Receiver Stages", fontsize=12)
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.ylim(min(cumulative_results) - 10, max(cumulative_results) + 10)
    
    plt.tight_layout()
    plt.show()

# --- UI Controls (5 Stages) ---
def create_stage(name, g_init, i_init):
    return [
        widgets.IntSlider(min=-20, max=30, value=g_init,step=0.1, description=f'{name} G (dB)'),
        widgets.IntSlider(min=-10, max=100, value=i_init,step=0.1, description=f'{name} IIP3 (dBm)')
    ]

s1 = create_stage('S1', -2.3, 100)  # Passive Filter
s2 = create_stage('S2', 20.5, 16.5)   # LNA
s3 = create_stage('S3', -7.3, 24.8)  # Mixer
s4 = create_stage('S4', -3.5, 100)  # IF Amp
s5 = create_stage('S5', 22, 15.3)  # ADC Driver

ui = VBox([
    HBox([s1[0], s1[1]]), HBox([s2[0], s2[1]]),
    HBox([s3[0], s3[1]]), HBox([s4[0], s4[1]]),
    HBox([s5[0], s5[1]])
])

params = {
    'g1': s1[0], 'i1': s1[1], 'g2': s2[0], 'i2': s2[1],
    'g3': s3[0], 'i3': s3[1], 'g4': s4[0], 'i4': s4[1],
    'g5': s5[0], 'i5': s5[1]
}

out = interactive_output(update_display, params)
display(ui, out)

Output()